### Imports

In [11]:
import os
import re
from typing import Dict

import mne
import numpy as np
import scipy.stats as sp_stats
from joblib import Parallel, delayed
from NirsLabProject.config.paths import Paths

from NirsLabProject.utils import pipeline_utils as putils
from NirsLabProject.config.consts import *

from NirsLabProject.utils import general_utils as utils
from NirsLabProject.config.consts import *
from NirsLabProject.config.subject import Subject
from NirsLabProject.utils.group_spikes import group_spikes
from NirsLabProject.utils import plotting


### Extract Paths of stimuli features
These are the features from the raw edf data without grouping, stimuli and sleeping features

In [2]:
subjects = [Subject(d, True) for d in os.listdir(Paths.products_data_dir_path) if d.startswith('p')]
stimuli_subjects = subjects
stimuli_subjects_with_features = filter(lambda subj: os.path.exists(subj.paths.subject_channels_spikes_features_path), stimuli_subjects)
stimuli_subjects_with_features = sorted(stimuli_subjects_with_features, key=lambda s: s.p_number)

In [23]:
def edit_coordinates(channels_spikes_features):
    channel_name_to_coordinates = utils.calculate_coordinates(subj)
    for channel, data in channels_spikes_features.items():
        x, y, z = utils.get_coordinates_of_channel(channel_name_to_coordinates, channel)
        data[:, CORD_X_INDEX] = x
        data[:, CORD_Y_INDEX] = y
        data[:, CORD_Z_INDEX] = z
    return channels_spikes_features

def regenerate_flat_features(subj: Subject):
    channels_spikes_features = np.load(
        subj.paths.subject_channels_spikes_features_path,
        allow_pickle=True
    ).item()
#     channels_spikes_features = edit_coordinates(channels_spikes_features)
    index_to_channel = putils.get_index_to_channel(subj, channels_spikes_features)
    groups, flat_features, _ = putils.group_spikes(subj, channels_spikes_features, index_to_channel)
    flat_features = utils.add_flag_of_scalp_detection_to_spikes_features(flat_features, None)
    flat_features = utils.add_stimuli_flag_to_spikes_features(subj, flat_features)
    flat_features = utils.add_sleeping_stage_flag_to_spike_features(subj, flat_features)
    subject_id = np.ones((flat_features.shape[0], 1)) * subj.p_number
    flat_features[:, SUBJECT_NUMBER] = subject_id.reshape(-1)
    return flat_features

for subj in stimuli_subjects_with_features:
    if subj.name in ['p485', 'p486', 'p488', 'p496', 'p499', 'p520']:
        flat_features = regenerate_flat_features(subj)
        print(subj.name, flat_features.shape)
        np.save(subj.paths.subject_flat_features_path, flat_features)
    

Grouping spikes
p485 (19202, 22)
Grouping spikes
p486 (24306, 22)
Grouping spikes
p488 (86479, 22)
Grouping spikes
p496 (16361, 22)
Grouping spikes
p499 (58390, 22)
Grouping spikes
p520 (1011, 22)


In [9]:
import shutil
import os

for subj in stimuli_subjects_with_features:
    new_path = '/tmp/flat_features'
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    shutil.copyfile(subj.paths.subject_flat_features_path, os.path.join(new_path, f'{subj.p_number}_flat_features.npy'))

In [22]:
_sum = 0
count = 0
for k,v in {
    'p485': 19202/30022,
    'p486': 24306/28050,
    'p488': 86479/111063,
    'p496': 16361/19573,
    'p499': 58390/66926,
    'p520': 1011/1390,
}.items():
    print(k,v)
    _sum += v
    count+= 1
    
print('avg:', _sum/count)


p485 0.6395976284058357
p486 0.866524064171123
p488 0.7786481546509638
p496 0.835896387881265
p499 0.8724561455936407
p520 0.7273381294964029
avg: 0.7867434183665384
